In [ ]:
import geopandas as gpd
import pandas as pd
import json
import os
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

import rasterio
from shapely.geometry import box, mapping
import matplotlib.pyplot as plt
import rasterio.plot  
from shapely.ops import transform
from pyproj import CRS
import contextily as ctx

custom functions
import sys
sys.path.append('../')
from utils.tiling import split_geotiff, reproject, split_annotations_to_geojson





In [ ]:
### 1. Split geotiff args ###

raster_folder = Path("../Satellite/big_tiles")
tile_output_folder = Path("../Satellite/small_tiles")

#make output diretctory if does not exist
if not os.path.exists(tile_output_folder):
    os.makedirs(tile_output_folder)

#details of small tiles
width = 1000 #width of the new tiles
height = 1000 #height of the new tiles
nso_pix = 1 #pixel size of the raster

### 2. Reproject geodataframe of a annotation shapefile args ###

annotations_crs = "epsg:4326"
raster_crs = "epsg:3857"


### 3.Spit annotations args ###

# Make folder for geojsons
json_folder = Path("../Satellite/geojsons")
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

In [ ]:
### 1. Spliting a list of raster into several tiles of x width and y height skipping black tiles ###
##Find this function in utils --> tiling.py 

#loop through the big tiles (rasters) and split them into smaller tiles
split_geotiff(raster_folder, tile_output_folder, width, height, nso_pix)



In [ ]:
### Reproject the DataFrame of an annotation CRS to the target CRS as the raster tiles ###

# Define the directory containing your GeoJSON files
annotations_path = '../Satellite/geojsons'
output_path = '../Satellite/geojsons_converted'

# Define the source and target CRS
annotations_crs = "epsg:4326"
raster_crs = "epsg:3857"

for filename in os.listdir(annotations_path):
        if filename.endswith(".geojson"):
            file_path = os.path.join(annotations_path, filename)
            gdf = gpd.read_file(file_path)

            if gdf.crs is None:
                gdf.set_crs(annotations_crs, inplace=True)
            
            # Ensuring type information is present
            if 'type' in gdf.columns:
                gdf = gdf.to_crs(raster_crs)
                output_file_path = os.path.join(output_path, f"reprojected_{filename}")
                gdf.to_file(output_file_path, driver='GeoJSON')
            else:
                print(f"No 'type' column found in {filename}")

print("Reprojection completed.")


In [ ]:
### Put all the geojsons in one dataframe


def load_geojson_files(directory, target_crs='EPSG:4326'):
    """
    Load all GeoJSON files from a directory into a single GeoDataFrame,
    ensuring a consistent CRS.

    Args:
    directory (str): Path to the directory containing the GeoJSON files.
    target_crs (str): The target CRS to which all GeoDataFrames will be transformed.

    Returns:
    GeoDataFrame: A GeoDataFrame containing all the features from the GeoJSON files.
    """
    geojson_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.geojson')]
    geojson_dfs = []
    
    for geojson_file in geojson_files:
        gdf = gpd.read_file(geojson_file)
        gdf = gdf.to_crs(target_crs)
        geojson_dfs.append(gdf)
    
    combined_gdf = gpd.GeoDataFrame(pd.concat(geojson_dfs, ignore_index=True))
    return combined_gdf

# Path to the directory containing the GeoJSON files
geojson_directory = '../Satellite/geojsons'

# Load the GeoJSON files into a single GeoDataFrame
df = load_geojson_files(geojson_directory)
print(df)


In [ ]:
### Split annotations DataFrame into the new small tiles of width X and height Y & save it to geojson ###

def split_geojson_annotations(tiles_directory, annotations_directory, output_directory, pixel_size):
    """
    This function splits the annotations in .geojson files to match the pre-split tiles.

    Parameters:
    tiles_directory (str): File path to the folder with the pre-split raster tiles.
    annotations_directory (str): File path to the folder with the annotations (.geojson).
    output_directory (str): File path to the folder where the split annotations will be saved.
    pixel_size (float): Pixel size of the raster.

    Returns:
    tuple: DataFrames with and without annotations, and a list of annotation files.
    """

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    annotation_files = [f for f in os.listdir(annotations_directory) if f.endswith('.geojson')]
    tiles_with_annotations = []
    tiles_without_annotations = []

    # Process each tile with a single progress bar
    tile_filenames = [f for f in os.listdir(tiles_directory) if f.endswith('.tif')]
    for tile_filename in tqdm(tile_filenames, desc="Processing tiles"):
        tile_path = os.path.join(tiles_directory, tile_filename)
        with rasterio.open(tile_path) as tile:
            annotations_found = False

            # Loop through each annotation file
            for annotation_filename in annotation_files:
                annotations_path = os.path.join(annotations_directory, annotation_filename)
                annotations = gpd.read_file(annotations_path)

                if annotations.crs != tile.crs:
                    transformed_annotations = annotations.to_crs(tile.crs)
                else:
                    transformed_annotations = annotations

                tile_geom = box(*tile.bounds)
                tile_gdf = gpd.GeoDataFrame([1], geometry=[tile_geom], crs=tile.crs)
                intersection = gpd.overlay(transformed_annotations, tile_gdf, how='intersection')

                if not intersection.empty:
                    annotations_found = True

                    # Prepare intersected annotations for saving
                    new_features = []
                    for _, row in intersection.iterrows():
                        new_feature = {
                            "type": "Feature",
                            "geometry": mapping(row['geometry']),
                            "properties": row.drop('geometry').to_dict()
                        }
                        new_features.append(new_feature)

                    new_annotations = {
                        "type": "FeatureCollection",
                        "features": new_features
                    }

                    output_annotation_path = os.path.join(output_directory, "{}.geojson".format(tile_filename.split('.')[0]))

                    #print(f"Creating annotation file: {output_annotation_path}")  # Debug statement

                    with open(output_annotation_path, 'w') as out_f:
                        json.dump(new_annotations, out_f, indent=2)

                    break  # Stop checking other annotation files if one match is found

            if annotations_found:
                tiles_with_annotations.append(tile_filename)
            else:
                tiles_without_annotations.append(tile_filename)

    # Create DataFrames from the lists
    df_annot = pd.DataFrame(tiles_with_annotations, columns=['Tile_Name'])
    df_empty_annot = pd.DataFrame(tiles_without_annotations, columns=['Tile_Name'])

    # Optionally, print the DataFrames
    print("Tiles with annotations:")
    print(df_annot)
    print("Tiles without annotations:")
    print(df_empty_annot)

    return df_annot, df_empty_annot, annotation_files

# Define your paths
tiles_directory = '../Satellite/small_tiles'  
annotations_directory = '../Satellite/geojsons'  
output_directory = '../Satellite/split_geojsons'  
pixel_size = 1.0  

# Run the function and store DataFrames
df_annot, df_empty_annot, annotation_files = split_geojson_annotations(tiles_directory, annotations_directory, output_directory, pixel_size)

# Display the first few rows of each DataFrame
print("DataFrame with no annotations:")
print(df_empty_annot.head())
print("\nDataFrame with annotations:")
print(df_annot.head())



In [ ]:
### Plot a small tile with its corresponding annotation

def plot_tile_with_annotation(tile_path, annotation_path):
    if not os.path.exists(tile_path):
        print(f"Tile not found: {tile_path}")
        return
    if not os.path.exists(annotation_path):
        print(f"Annotation not found: {annotation_path}")
        return
    
    with rasterio.open(tile_path) as tile:
        fig, ax = plt.subplots(figsize=(10, 10))
        rasterio.plot.show(tile, ax=ax)
        annotations = gpd.read_file(annotation_path)
        annotations.plot(ax=ax, facecolor='none', edgecolor='red')
        plt.show()

if not df_annot.empty:
    example_tile = os.path.join(tiles_directory, df_annot['Tile_Name'].iloc[0])
    annotation_filename = "{}.geojson".format(df_annot['Tile_Name'].iloc[0].split('.')[0])
    example_annotation = os.path.join(output_directory, annotation_filename)
    print(f"Example tile path: {example_tile}")
    print(f"Example annotation path: {example_annotation}")
    plot_tile_with_annotation(example_tile, example_annotation)
else:
    print("No tiles with annotations found.")



In [ ]:
###Extract labels of land uses and print unique types 

def print_unique_types(annotations_path):
    unique_types = set()  # Using a set to avoid duplicate types

    # Check all GeoJSON files in the given directory
    for filename in os.listdir(annotations_path):
        if filename.endswith(".geojson"):
            file_path = os.path.join(annotations_path, filename)
            try:
                # Load the GeoJSON file into a GeoDataFrame
                gdf = gpd.read_file(file_path)
                
                # Check if 'type' column exists in the GeoDataFrame
                if 'type' in gdf.columns:
                    # Update the set with unique types found in this file
                    unique_types.update(gdf['type'].unique())
                else:
                    print(f"No 'type' column found in {filename}")
            except Exception as e:
                print(f"Failed to process {filename}: {e}")

    # Print all unique types collected from all files
    print("Unique types found:")
    for typ in unique_types:
        print(typ)

annotations_path =  Path("../Satellite/geojsons_test")
print_unique_types(annotations_path)


In [ ]:
### Print the locations of the ports on a map 

# Define the directory where your GeoJSON files are located
directory = '../Satellite/geojsons_converted'
target_crs = 'EPSG:3857'  # Web Mercator for basemap compatibility

# Initialize an empty list to store the GeoDataFrames
geojson_gdfs = []

# Loop through the files in the directory and read the GeoJSON files
for filename in os.listdir(directory):
    if filename.endswith(".geojson"):
        file_path = os.path.join(directory, filename)
        gdf = gpd.read_file(file_path)
        if gdf.crs != target_crs:
            gdf = gdf.to_crs(target_crs)
        geojson_gdfs.append(gdf)

# Concatenate all the GeoDataFrames into a single GeoDataFrame
all_ports_gdf = pd.concat(geojson_gdfs, ignore_index=True)
all_ports_gdf = gpd.GeoDataFrame(all_ports_gdf, crs=target_crs)

# Ensure all geometries are Points (use centroids for polygons)
all_ports_gdf['geometry'] = all_ports_gdf.geometry.centroid

# Load Europe map and convert to the same CRS as the basemap
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world[(world.continent == "Europe")].to_crs(target_crs)

# Plot the map
fig, ax = plt.subplots(figsize=(15, 10))
europe.boundary.plot(ax=ax, linewidth=1, edgecolor='black')
all_ports_gdf.plot(ax=ax, color='red', markersize=50, alpha=0.6, label='Ports')

# Set the extent to focus on Europe in Web Mercator projection
ax.set_xlim(-5000000, 5900000)
ax.set_ylim(3000000, 11000000)

# Add a basemap
try:
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
except Exception as e:
    print(f"Error adding basemap: {e}")

# Add gridlines and labels
ax.grid(True, linestyle='--', alpha=0.7)
ax.set_title('Ports on the Map of Europe', fontsize=15, fontweight='bold')
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

# Add a legend
ax.legend(loc='lower left', fontsize=12)

# Improve layout and show the plot
plt.tight_layout()
plt.show()


In [ ]:
###Looks for which files contain a certain type (for example 'refinery'), I used this to visually examine the tiles 

# def get_files_with_refinery_annotation(geojson_dir, target_type='refinery'):
#     files_with_refinery = []

#     for filename in os.listdir(geojson_dir):
#         if filename.endswith('.geojson'):
#             file_path = os.path.join(geojson_dir, filename)
#             with open(file_path) as f:
#                 data = json.load(f)

#             for feature in data['features']:
#                 if 'properties' in feature and 'type' in feature['properties']:
#                     if feature['properties']['type'] == target_type:
#                         files_with_refinery.append(filename)
#                         break  # Move to the next file once we find 'refinery' in the current one

#     return files_with_refinery 

# # Example usage
# geojson_dir = "../Satellite/geojsons"
# files_with_refinery = get_files_with_refinery_annotation(geojson_dir)
# print("Files with 'refinery' annotation:", files_with_refinery)